In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version


# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-09 06:06:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.23MB/s    in 0.7s    

2022-06-09 06:06:22 (1.23 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Pet Productdataset
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [5]:
#Count initial records in dataset
print(f'There are {df.count()} records')

There are 2642434 records


In [6]:
#Drop NA and count records in dataset
df = df.dropna()
print(f'There are {df.count()} records after dropping NAs')

There are 2642195 records after dropping NAs


In [7]:
#Drop duplicates and count records in dataset
df = df.dropDuplicates()
print(f'There are {df.count()} records after dropping duplicates')

There are 2642195 records after dropping duplicates


In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [21]:
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType,DateType

In [9]:
review_id_table_df = df.select(['review_id','customer_id','product_id','product_parent','review_date'])

In [23]:
review_id_table_df.withColumn("review_id", review_id_table_df.review_id.cast(StringType())).withColumn("customer_id", review_id_table_df.customer_id.cast(StringType())).withColumn("product_id", review_id_table_df.product_id.cast(StringType())).withColumn("product_parent", review_id_table_df.product_parent.cast(StringType())).withColumn("review_date", review_id_table_df.review_date.cast(DateType()))

DataFrame[review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: date]

In [10]:
review_id_table_df.count()

2642195

In [11]:
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100E40KO3UFDH|   19431883|B00KPSWRCQ|      79805312| 2015-03-09|
|R101DIHNYOAWOU|    1410253|B005G520L6|     326063757| 2014-07-15|
|R101RUFT13ZT8M|   13174422|B00HYNCO0U|     329336165| 2015-03-07|
|R1028MTZVYMH4L|   52268907|B00CT073WM|     216674350| 2013-11-21|
|R102J5WM6GQCSU|   10095195|B00LOQ3YDY|     448212337| 2015-02-12|
|R102N3ZSEBW4II|   51131561|B00005QY24|     231082442| 2002-07-07|
|R102ZOAL7RH9YA|    1539319|B004GEHTVY|     629315554| 2014-08-08|
|R10385HHG2048Q|   16579915|B00NOD0GX6|      25350627| 2015-04-06|
|R1042I45IIWNLG|    6534390|B00006IFIM|     391841053| 2014-11-16|
|R104FVJ62YQKTW|    8577338|B00BQGL4ZW|     740703956| 2015-03-12|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [24]:
products_df = df.select(['product_id','product_title'])


In [13]:
products_df.count()

2642195

In [25]:
products_df = products_df.dropDuplicates()

In [26]:
products_df.count()

313238

In [16]:
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00PITWBPA|Teacher Lesson Pl...|
|B0038ZTDGQ|        LINKYO TN650|
|B0028N6PSI|Mead Composition ...|
|B001IZX8AA|Generic DR400 and...|
|B010B1ROR4|Viz-pro Double-si...|
|B00WT9IDPE|AIM Compatible Re...|
|B003ZZUKPC|HTS 262B1 1,000 P...|
|B00NQ3UB34|Plush Cute Genuin...|
|B00JFOJ7PG|BIC Kids Pencil -...|
|B000AN1Q64|Pendaflex Hanging...|
+----------+--------------------+
only showing top 10 rows



In [27]:
tempdf = df.select(['customer_id'])

In [28]:
tempdf2 = tempdf.groupby('customer_id').agg({'customer_id':'count'})

In [29]:
customers_df = tempdf2.withColumnRenamed("count(customer_id)", "customer_count")

In [30]:
customers_df.count()

1790107

In [32]:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [33]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [34]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)